# Aula 17: ACID

# Notebook A

Vamos explorar algumas características de transações em bancos de dados relacionais. Estas características são representadas pela sigla **ACID**:

- **Atomicity**: em uma transação, um conjunto de comandos é executado ou rejeitado como uma única unidade.

- **Consistency**: o banco de dados passa de um estado válido para outro estado válido a cada transação.

- **Isolation**: várias transações concorrentes podem acontecer sem que uma transação interfira diretamente na outra, ou seja, o usuário tem a percepção de que as transações foram executadas sequencialmente.

- **Durability**: uma transação confirmada permanecerá gravada mesmo que a energia acabe ou o sistema trave.

Para testar algumas dessas propriedades, vamos instalar a nossa base `tranqueira`, a base de dados de perigos alimentícios! Rode o script `tranqueira.sql`, cujo conteúdo está descrito abaixo:

```sql
DROP DATABASE IF EXISTS tranqueira;
CREATE DATABASE tranqueira;
USE tranqueira;

CREATE TABLE comida (
    id INT NOT NULL AUTO_INCREMENT,
    Nome VARCHAR(30),
    idPerigo INT,
    PRIMARY KEY (id)
);

CREATE TABLE perigo (
    id INT NOT NULL AUTO_INCREMENT,
    Nome VARCHAR(20),
    PRIMARY KEY (id)
);

ALTER TABLE comida ADD CONSTRAINT fk_perigo FOREIGN KEY (idPerigo) REFERENCES perigo (id);

INSERT INTO perigo VALUES (1, 'Cardiaco'), (2, 'Intestinal'), (3, 'Dermatologico'), (4, 'Mental');
INSERT INTO comida VALUES (1, 'Torresmo', 1), (2, 'Alface', NULL), (3, 'Coxinha', 2), (4, 'Espetinho', 2);

SELECT * FROM comida;
SELECT * FROM perigo;
```

<table>
    <tr>
        <th> Tabela comida </th>
        <th> Tabela perigo </th>
    </tr>
    <tr>
        <td><img src="comida.png"/></td>
        <td><img src="perigo.png"/></td>
    </tr>
</table>

Nesta aula vamos trabalhar com dois notebooks simultaneamente, para observar o que acontece quando dois processos concorrentes acessam a mesma base de dados.

In [ ]:
import mysql.connector
import os
from functools import partial
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
script = """
DROP DATABASE IF EXISTS tranqueira;
CREATE DATABASE tranqueira;
USE tranqueira;

CREATE TABLE comida (
    id INT NOT NULL AUTO_INCREMENT,
    Nome VARCHAR(30),
    idPerigo INT,
    PRIMARY KEY (id)
);

CREATE TABLE perigo (
    id INT NOT NULL AUTO_INCREMENT,
    Nome VARCHAR(20),
    PRIMARY KEY (id)
);

ALTER TABLE comida ADD CONSTRAINT fk_perigo FOREIGN KEY (idPerigo) REFERENCES perigo (id);

INSERT INTO perigo VALUES (1, 'Cardiaco'), (2, 'Intestinal'), (3, 'Dermatologico'), (4, 'Mental');
INSERT INTO comida VALUES (1, 'Torresmo', 1), (2, 'Alface', NULL), (3, 'Coxinha', 2), (4, 'Espetinho', 2);
"""

connection = mysql.connector.connect(
        host=os.getenv('MD_DB_SERVER'),
        user=os.getenv('MD_DB_USERNAME'),
        password=os.getenv('MD_DB_PASSWORD'),
    )

with connection.cursor() as cursor:
    try:
        for _ in cursor.execute(script, multi=True):
            pass
        cursor.execute("COMMIT")
    except Exception as e:
        print(e)
        cursor.execute("ROLLBACK")

connection.close()

In [ ]:
def get_connection_helper(database):

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print("Executando query:")
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host=os.getenv('MD_DB_SERVER'),
        user=os.getenv('MD_DB_USERNAME'),
        password=os.getenv('MD_DB_PASSWORD'),
        database=database,
    )
    return connection, partial(run_db_query, connection)


connection1, db1 = get_connection_helper("tranqueira")

In [ ]:
db1("SELECT * FROM comida")
db1("SELECT * FROM perigo")

<img src="pare.png" width=150px/>

Passe para o notebook B e abra a conexão `db2`, ponto 1, volte para cá em seguida. Dica: procure descobrir o *shortcut* para trocar de aba do browser rapidamente. No Chrome é Ctrl-PgUp / Ctrl-PgDn.

Temos duas conexões abertas agora, em processos diferentes: `db1` e `db2`. Vamos trabalhar alternadamente com elas para observar efeitos de concorrência.

## Níveis de isolamento

Vamos explorar os 4 níveis de isolamento definidos pelo padrão ANSI/SQL:

- SERIALIZABLE

- REPEATABLE READ

- READ COMMITED

- READ UNCOMMITED


## `SERIALIZABLE`

Vamos colocar o nível de isolamento em ambos os notebooks em `SERIALIZABLE`. 

In [ ]:
db1("SET SESSION TRANSACTION ISOLATION LEVEL SERIALIZABLE")

Este comando indica que todas as transações subsequentes desta sessão terão nível de isolamento `SERIALIZABLE`. Se ao invés de `SESSION` usarmos `GLOBAL`, esta mudança de nível de isolamento vai valer para todas as sessões deste banco de dados. Somente root ou algum super-usuário pode executar esse comando com atributo `GLOBAL`.

Qualquer query equivale a um início de transação. Se desejamos marcar um ponto específico como início de transação, devemos fazê-lo explicitamente com `START TRANSACTION`.

Execute a célula abaixo:

In [ ]:
db1("START TRANSACTION")

<img src="pare.png" width=150px/>

Vá para o ponto 2 no notebook B.

---

Execute a query abaixo:

In [ ]:
db1("SELECT * FROM perigo")
db1('INSERT INTO perigo(Nome) VALUES ("Econômico")')
db1("SELECT * FROM perigo")

<img src="pare.png" width=150px/>

Não prossiga por aqui! Agora vá para o notebook B, ponto 3.

---

Ao retornar do notebook B rode a célula abaixo para terminar a transação.

In [ ]:
db1("COMMIT")

<img src="pare.png" width=150px/>

Volte para o notebook B e observe o resultado.

---

Agora que você retornou de B, rode a célula abaixo, o que vai acontecer?

In [ ]:
db1("SELECT * FROM perigo")
db1("COMMIT")

Você observará o mesmo fenômeno de *timeout* que aconteceu em B! Por que isso aconteceu? Porque a transação de B não terminou! Volte para o notebook B e siga as instruções para finalizar a transação. Em seguida volte para cá e rode de novo esse `SELECT`.

<img src="pare.png" width=150px/>

Agora sim funcionou!

---

Como vocês podem ver o nível `SERIALIZABLE` é muito restritivo, e pode causar *deadlocks* facilmente.

*Vamos pensar um pouco*: o que aconteceria se não existisse *timeout* e tentássemos executar, concorrentemente, os seguintes códigos:

<table>
    <tr>
        <th>Sessão A</th>
        <th>Sessão B</th>
    </tr>
    <tr>
        <td>`START TRANSACTION;`</td>
        <td></td>
    </tr>
    <tr>
        <td></td>
        <td>`START TRANSACTION;`</td>
    </tr>
    <tr>
        <td>`SELECT * FROM perigo;`
        </td>
        <td></td>
    </tr>
    <tr>
        <td></td>
        <td>`SELECT * FROM comida;`</td>
    </tr>
    <tr>
        <td>`SELECT * FROM comida;`</td>
        <td></td>
    </tr>
    <tr>
        <td></td>
        <td>`SELECT * FROM perigo;`</td>
    </tr>
    <tr>
        <td>`ROLLBACK;`</td>
        <td></td>
    </tr>
    <tr>
        <td></td>
        <td>`ROLLBACK;`</td>
    </tr>
</table>

Quais queries seriam executadas, e quais iriam ficar travadas?

<div class="alert alert-success">

Sua resposta AQUI!

</div>

---

Se o banco de dados estiver travado e você tiver que cancelar uma query de algum usuário: abra o MySQL Workbench e nele você encontrará a aba "Client Connections". Nela você poderá cancelar uma query.

![Tela kill query](kill_query.png)

## `REPEATABLE READ`

Vamos mudar o nível de isolamento para `REPEATABLE READ`, o padrão do MySQL:

In [ ]:
db1("SET SESSION TRANSACTION ISOLATION LEVEL REPEATABLE READ;")

<img src="pare.png" width=150px/>

Vá para o notebook B, ponto 4.

---

Agora vamos repetir algumas das atividades que fizemos no caso anterior. Vamos rodar a célula abaixo:

In [ ]:
db1("START TRANSACTION")

<img src="pare.png" width=150px/>

Vá para o ponto 5 do notebook B.
 
---

Novamente, vamos ver os valores da tabela `perigo`:

In [ ]:
db1("SELECT * FROM perigo")
db1('INSERT INTO perigo(Nome) VALUES ("Emocional")')
db1("SELECT * FROM perigo")

<img src="pare.png" width=150px/>

Passe para o notebook B, ponto 6.

---

Rode de novo o `SELECT`:

In [ ]:
db1("SELECT * FROM perigo")

Observe que o `SELECT` continua retornando os mesmos valores. 

<img src="pare.png" width=150px/>

Vá para o notebook B, ponto 7, e feche a transação.

---

Rode de novo o `SELECT`:

In [ ]:
db1("SELECT * FROM perigo")

In [ ]:
db1("ROLLBACK")

O MySQL previne *phantom reads*: enquanto a transação não terminar, vale o mesmo resultado do `SELECT`. Isso é uma peculiaridade do MySQL, outros bancos de dados podem não ter a mesma natureza. Como tudo no mundo SQL, a padronização é sofrível...

(Um link interessante: https://blog.pythian.com/understanding-mysql-isolation-levels-repeatable-read/)

<img src="pare.png" width=150px/>

Agora vá para o notebook B, ponto 8.

---

Teste novamente o `SELECT`:

In [ ]:
db1("SELECT * FROM perigo")

Esta sessão ainda não enxergou as mudanças da sessão vizinha! Vamos encerrar a transação e tentar de novo.

In [ ]:
db1("COMMIT")
db1("SELECT * FROM perigo")

Agora sim conseguimos enxergar o estado atual do banco de dados!

Por fim, vamos fechar as conexões. Rode a célula abaixo, depois vá para o notebook B, ponto 9, e faça o mesmo.

In [ ]:
connection1.close()

Isolamento de transações é um tópico complicado em bancos de dados. Apesar dos esforços de padronização, cada banco de dados pode implementar de modo ligeiramente diferente os níveis de isolamento. Até mesmo dentro do mesmo banco de dados podemos ter *engines* diferentes com implementações diferentes dos níveis de isolamento! Para o MySQL e seu *engine* InnoDB, verifique a documentação em https://dev.mysql.com/doc/refman/8.0/en/innodb-transaction-isolation-levels.html

Problemas de concorrência são algumas das questões mais complicadas em computação. Vocês verão em mais detalhes na disciplina "Supercomputação" como lidar com alguns problemas clássicos em concorrência. Mais ainda, lidar com concorrência em um ambiente distribuído e não-confiável é um problema muito maior, razão pela qual muitos bancos de dados distribuídos NoSQL não seguem as características **ACID**, mas sim apresentam o comportamento **BASE**:

- **B**asically **A**vailable: disponibilidade resultante de replicação de recursos;
- **S**oft state: Não temos garantias de consistência, isso fica à cargo da aplicação do usuário;
- **E**ventually consistent: eventualmente as modificações que aconteceram em um nó são propagadas para o resto da rede - no longo prazo a base será consistente, mas no curto prazo podem existir dados "velhos".

Ou seja, são bases de dados que aceitam inconsistência temporária. Isso é consequência do teorema CAP: "**C**onsistency, **A**vailability, **P**artition tolerance: pick two".

Leituras recomendadas: 

- Chandra, D. G. "BASE analysis of NoSQL database". Future Generation Computer Systems, 52 (2015), pp. 13–21.

- Corbellini, A., Mateos, C., Zunino, A., Godoy, D., Schiaffino, S. "Persisting big-data: The NoSQL landscape". Information Systems, 63 (2017), pp. 1–23.